In [1]:
import pandas as pd
import numpy as np
import re
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from bs4 import BeautifulSoup
import requests
from nltk.tokenize import word_tokenize
from datetime import datetime

In [2]:
movie_data  = pd.read_csv(r'E:\csv_datas\tmdb_movie_data\tmdb_5000_movies.csv')
movie_data.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124


In [3]:
movie_data.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [4]:
release_date = movie_data.release_date

In [5]:
column = ['title', 'genres', 'overview', 'vote_average', 'vote_count']

In [6]:
# movie_data = movie_data[column]
# movie_data.to_csv(r'E:\csv_datas\tmdb_movie_data\e.csv')
# # movie_data.head()

In [7]:
movie_data = pd.read_csv(r'E:\csv_datas\tmdb_movie_data\a.csv')
movie_data = movie_data[column]
movie_data.head()

,title,genres,overview,vote_average,vote_count
0,Avatar,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","In the 22nd century, a paraplegic Marine is di...",7.2,11800
1,Pirates of the Caribbean: At World's End,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...","Captain Barbossa, long believed to be dead, ha...",6.9,4500
2,Spectre,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",A cryptic message from Bond’s past sends him o...,6.3,4466
3,The Dark Knight Rises,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",Following the death of District Attorney Harve...,7.6,9106
4,John Carter,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...","John Carter is a war-weary, former military ca...",6.1,2124


In [8]:
def genre_edit():
    genres = []
    for i in movie_data.genres:
        genre = re.sub("[][]", "", i)
        genre = genre+', '
        genre = genre.split("}, ")
        for j in range(len(genre)):
            a = genre[j]+'}'
            if a == "}":
                break
            a = json.loads(a)
            genre[j] = a['name']
        genre = " ".join(genre)
        genres.append(genre)
    return genres

In [9]:
genres = genre_edit()

In [10]:
movie_data.genres = genres
# movie_data.to_csv(r'E:\csv_datas\tmdb_movie_data\b.csv')


In [11]:
movie_data.head()

,title,genres,overview,vote_average,vote_count
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",7.2,11800
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",6.9,4500
2,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,6.3,4466
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,7.6,9106
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",6.1,2124


In [12]:
movie_data.isna().sum()

title           0
genres          0
overview        0
vote_average    0
vote_count      0
dtype: int64

In [13]:
movie_data['title'].duplicated().sum()

3

In [14]:
duplicate_rows = movie_data.loc[movie_data['title'].duplicated()]
duplicate_rows

,title,genres,overview,vote_average,vote_count
2877,The Host,Horror Drama Science Fiction,Gang-du is a dim-witted man working at his fat...,6.7,537
3693,Out of the Blue,Drama,Ordinary people find extraordinary courage in ...,5.9,18
4267,Batman,Family Adventure Comedy Science Fiction Crime,The Dynamic Duo faces four super-villains who ...,6.1,203


In [15]:
movie_data = movie_data.drop(duplicate_rows.index)

In [16]:
movie_data['title'].duplicated().sum()

0

In [ ]:
movie_data.reset_index(inplace=True, drop=True)

In [18]:
movie_data['release_date'] = release_date

In [4]:
# movie_data.to_csv(r'E:\csv_datas\tmdb_movie_data\d.csv')
movie_data = pd.read_csv(r'E:\csv_datas\tmdb_movie_data\d.csv')

In [27]:
# movie_data.drop(['Unnamed: 0 '], axis=1)

In [5]:
movie_data.head()

,Unnamed: 0,title,genres,overview,vote_average,vote_count,release_date
0,0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",7.2,11800,2009-12-10
1,1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",6.9,4500,2007-05-19
2,2,Spectre,Action Adventure Crime,A cryptic message from Bond’s past sends him o...,6.3,4466,2015-10-26
3,3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,7.6,9106,2012-07-16
4,4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",6.1,2124,2012-03-07


In [23]:
movie_index = pd.Series(movie_data.index, movie_data['title'].str.lower())

In [24]:
def overview_classifier(title):
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(movie_data.overview)
    cosine_score = linear_kernel(tfidf_matrix, tfidf_matrix)
    index = movie_index[title.lower()]
    movie_scores = cosine_score[index]
    movie_scores = list(enumerate(movie_scores))
    movie_scores_sorted = sorted(movie_scores, key=lambda x: x[1], reverse=True)
    top_index = [i[0] for i in movie_scores_sorted]
    top_picks = movie_data.loc[movie_data.index[top_index[0:1001]]]
    
    return top_picks


In [25]:
def genre_classifier(data):
    data.reset_index(inplace=True, drop=True)
    tfidf = TfidfVectorizer()
    tfidf_matrix = tfidf.fit_transform(data.genres)
    cosine_score = linear_kernel(tfidf_matrix, tfidf_matrix)
    movie_scores = cosine_score[0]
    movie_scores = list(enumerate(movie_scores))
    movie_scores_sorted = sorted(movie_scores, key=lambda x:x[1], reverse=True)
    top_index = [i[0] for i in movie_scores_sorted]
    top10_movies = data.loc[data.index[top_index[1:11]]]
    return top10_movies
    
    

In [26]:
movie_name = input('enter movie name :')
top_picks = overview_classifier(movie_name)
top10_movies = genre_classifier(top_picks)

enter movie name :spectre


In [27]:
top10_movies.head()

,Unnamed: 0,title,genres,overview,vote_average,vote_count,release_date
5,11,Quantum of Solace,Adventure Action Thriller Crime,Quantum of Solace continues the adventures of ...,6.1,2965,2008-10-30
373,1082,16 Blocks,Action Adventure Crime Thriller,An aging cop is assigned the ordinary task of ...,6.2,661,2006-03-01
443,1450,Punisher: War Zone,Action Crime,Waging his one-man war on the world of organiz...,5.6,294,2008-12-05
652,3847,The Masked Saint,Crime Action,The journey of a professional wrestler who bec...,2.9,7,2006-02-07
785,2295,Firestorm,Action Crime,A crew of seasoned criminals led by the notori...,5.9,43,2013-12-12


In [28]:

# movie_data.to_csv(r'E:\csv_datas\tmdb_movie_data\top10.csv')


In [36]:
top10_movies.reset_index(inplace=True, drop=True)

In [37]:
url = "https://yts.mx/movies/"
def data_fetch(data):
    imdb_rating = []
    thumbnail_link = []
    for i in range(len(data)):
        name = data.title[i].lower()
        date = data.release_date[i]
        name = re.sub(r'[^a-z0-9 '']', "", name)
        name = word_tokenize(name)
        date = datetime.strptime(date, "%Y-%m-%d")
        date = date.year
        name.append(str(date))
        name = "-".join(name)
        page = requests.get(url+name)
        if page.status_code == 200:
            html = BeautifulSoup(page.content, 'lxml')
            rating = html.find_all("span", itemprop='ratingValue')
            imdb_rating.append(rating[0].text)
            link = html.find_all('img', attrs={'class': 'img-responsive'})
            link = link[0]['content']
            thumbnail_link.append(link)
        else:
            imdb_rating.append("--")
            thumbnail_link.append("--")
            
    data = {'imdb_rating': imdb_rating, 'thumnail_link': thumbnail_link}
    return data
            
             
        

In [ ]:
data1 = data_fetch(top10_movies)